# dummy classic ml model

In [1]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import joblib

X, y = make_regression(1000, 3)

model = LinearRegression().fit(X, y)

In [20]:
model.predict([[1,2,3]])

array([409.09602573])

In [21]:
joblib.dump(model, "model.joblib")

['model.joblib']

In [22]:
joblib.load("model.joblib")

LinearRegression()

In [21]:
import requests

r = requests.post(
    url="http://localhost:8080/classic",
    json={"a": 1, "b": 2, "c": 3}
)

r.json()

{'detail': 409.0960257291469}

In [18]:
for _ in range(10):
    r = requests.post(url="http://localhost:8080/classic", json={"a":2, "b": 2000, "c": 10})
    r.json()

# dummy LLM (BERT) model

In [1]:
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained("ai-forever/ruBert-base")
tokenizer = BertTokenizer.from_pretrained("ai-forever/ruBert-base", do_lower_case=True)

/Users/aweeu/Desktop/docker_llm_webinar/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
encodings = tokenizer.encode_plus(
    " ".join(["word"] * 10),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors="pt",
)
output = model(**encodings)
# output.pooler_output[0]

/Users/aweeu/Desktop/docker_llm_webinar/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [4]:
import torch

encodings = tokenizer.encode_plus(
    " ".join(["word"] * 10),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors="pt",
)
dummy_input = (encodings["input_ids"], encodings["attention_mask"], encodings["token_type_ids"]) # Пример входного тензора
onnx_file_path = "model.onnx"  # Путь для сохранения ONNX файла

# Экспорт модели
with torch.no_grad():
    torch.onnx.export(
        model,  # Модель PyTorch
        dummy_input,  # Пример входных данных
        onnx_file_path,  # Путь для сохранения ONNX файла
        export_params=True,  # Экспортировать обученные параметры
        opset_version=14,  # Версия ONNX операторов
        do_constant_folding=True,  # Оптимизация констант
        input_names=["input_ids", "attention_mask", "token_type_ids"],
        output_names=["pooler_output"],
        dynamic_axes={
            "input_ids": {0: "batch_size", 1: "seq_len"},
            "attention_mask": {0: "batch_size", 1: "seq_len"},
            "token_type_ids": {0: "batch_size", 1: "seq_len"},
            "pooler_output": {0: "batch_size"},
        },
    )

print(f"Модель успешно экспортирована в файл: {onnx_file_path}")

Модель успешно экспортирована в файл: model.onnx


In [17]:
import requests

r = requests.post(
    url="http://localhost:8080/predict",
    params={"model_name": "bert_model"},
    json={
        "text": "example of the text"
    }
)

r.json()

KeyboardInterrupt: 